In [1]:
import matplotlib.pyplot as plt

import snowflake.connector
import joblib
import scipy

import pandas as pd 
import numpy as np

from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
from sqlalchemy.dialects import registry

from azureml.core import Workspace, Dataset, Run
from azureml.core.model import Model


from azure.identity import ManagedIdentityCredential, InteractiveBrowserCredential, DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

from sentence_transformers import SentenceTransformer, util

from sklearn.metrics import pairwise_distances
from sklearn.manifold import TSNE, MDS
from sklearn.metrics.pairwise import cosine_distances
from scipy.spatial.distance import cosine
import torch
from sklearn.metrics import jaccard_score, multilabel_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from scipy.spatial import distance


from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer, sample_dataset

from datasets import Dataset
import evaluate
from sklearn.preprocessing import OrdinalEncoder

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.linear_model import LogisticRegression
import re

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (9.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
2023-09-12 12:44:12.592042: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-12 12:44:13.398761: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-09-12 12:44:13.398844: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library '

In [2]:
from umap import UMAP
from sklearn.metrics import classification_report

In [3]:
pd.options.display.max_columns = None
pd.set_option('display.max_row', 2500)

In [4]:
subscription_id = '75310040-ed56-4e53-83e0-80f23ef48cd6'
resource_group = 'dlsap-PaaS-PRD-rgp-001'
workspace_name = 'Finance_DS'

workspace = Workspace(subscription_id, resource_group, workspace_name)

In [5]:
credential = DefaultAzureCredential()
secret_client = SecretClient(vault_url="https://financeds.vault.azure.net/", credential=credential)
        
account = secret_client.get_secret("snowflake-account").value
user = secret_client.get_secret("snowflake-user").value
password = secret_client.get_secret("snowflake-password").value
database = secret_client.get_secret("snowflake-database").value
role = secret_client.get_secret("snowflake-role").value
warehouse = secret_client.get_secret("snowflake-warehouse").value

engine_1 = create_engine(URL(
            account = account,
            user = user,
            password = password,
            database = database,
            role = role,
            warehouse=warehouse,
            schema='sinergi'
        ))

query_1 = f""" select * from RW_ACTUALS_ALL_TR
        """
incidents = pd.read_sql(query_1, con=engine_1)




KeyboardInterrupt



In [6]:
incidents.head(3)

,caseno,company,functional_group,function,functional_area,functional_location,functional_sub_location,location_sid,location_short,country_short,sl_country,sl_location_lvl_1,sl_location_lvl_2,sl_location_lvl_3,sl_location_lvl_4,title,employment_category,case_type,case_severity,actual_workdays_lta,case_description,status,hazard_physical_security_event,work_process,imm_action_taken_recom,full_investigation_done,case_occurence_date,created_date,modified_date,approved_within_deadline,personal_injuries,case_closed_date,cases_no_of_registrations,valid_from,valid_to,case_description_en,language
0,32776,Uniper Group (70006651),COO (70017898),Asset Operations (70017927),CCGT (70017930),Grain UK (70019210),,8138,PP GRA,UK,UK,Gas Turbine Fleet,Grain,None,None,61BFT20 CB faulty,Uniper employee,Process Safety,Near Hits,NaN,415V circuit breaker from 11kv transformer sup...,Approved,Occupational Safety and Process Safety,Inspection and monitoring,Recommended to fault find why this has happene...,Investigated,2023-03-19,2023-03-20,2023-05-24,No,NaN,None,1.0,2023-05-25 01:50:46,2100-12-31,415V circuit breaker from 11kv transformer sup...,en
1,32024,Uniper Group (70006651),COO (70017898),Nuclear Sweden (70017948),PM Oskarshamn (70018698),,,8141,NPP OS,NO,Sweden,Nuclear Sweden,Oskarshamn (Nuclear),None,None,Oskarshamn 1 - RIO - Pardörr mellan 4.71 och 4...,Contractor,Safety,Observations,NaN,Oskarshamn 1 - RIO - Pardörr mellan 4.71 och 4...,Closed,Occupational Safety and Process Safety,-- Not selected --,None,Not investigated,2023-02-15,2023-02-15,2023-05-24,-- Not selected --,NaN,2023-03-17,1.0,2023-05-25 01:50:46,2100-12-31,Oskarshamn 1 - RIO - Pair door between 4.71 an...,sv
2,29905,Uniper Group (70006651),COO (70017898),Asset Operations (70017927),Hydro DE (70018324),Kraftwerksgruppe Lech (70018328),Betrieb & Instandhaltung (70018347),8279,FIS,DE,Germany,Hydro DE,Rivergroup Lech,Finsterau KW,None,Fluchtwegekennzeichnung,Uniper employee,None,Observations,NaN,Fluchtwegekennzeichnung Lagerraum Ausgangsbauw...,Closed,Lack of emergency routes,-- Not selected --,Kraftwerker eingewiesen und Anbringung angeord...,Not investigated,2022-11-21,2022-11-25,2023-05-24,No,NaN,2023-05-24,1.0,2023-05-25 01:50:46,2100-12-31,Escape route marking storage room starting str...,de


In [11]:
incidents = incidents.drop_duplicates(['case_description_en'], keep=False)
incidents = incidents[~incidents['case_type'].isna()]

In [12]:
texts = incidents['case_description_en'].values
targets = incidents['case_type'].values

In [13]:
texts_train, texts_test, targets_train, targets_test = train_test_split(texts, targets, test_size=0.15)

In [14]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
Y_train = enc.fit_transform(targets_train.reshape(-1, 1))
Y_test = enc.fit_transform(targets_test.reshape(-1, 1))

## 3:

In [25]:
model = SetFitModel.from_pretrained(
    'sentence-transformers/all-mpnet-base-v2')


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [26]:
ord_enc = OrdinalEncoder()
ord_enc.fit(targets_train.reshape(-1, 1))

targets_test_ = ord_enc.transform(targets_test.reshape(-1, 1))
targets_train_ = ord_enc.transform(targets_train.reshape(-1, 1))

In [27]:
targets_test_ = targets_test_.flatten()
targets_train_ = targets_train_.flatten()

In [28]:
eval_dataset = Dataset.from_dict({'text':  texts_test, 'label': targets_test_})
train_dataset = Dataset.from_dict({'text':  texts_train, 'label': targets_train_})

In [29]:
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    metric='accuracy',
    num_iterations=5,
    num_epochs=2,
    batch_size=8,
)



In [30]:
trainer.train()
metrics = trainer.evaluate()

Generating Training Pairs:   0%|          | 0/5 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 74840
  Num epochs = 2
  Total optimization steps = 18710
  Total train batch size = 8


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9355 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9355 [00:00<?, ?it/s]

***** Running evaluation *****


In [31]:
metrics

{'accuracy': 0.8039364118092355}

In [33]:
#model._save_pretrained('./model__/')
joblib.dump((texts_train, texts_test, targets_train, targets_test), './data_.pkl', compress=9)

['./data_.pkl']

### logistic regression 1:

In [34]:
model_ = SentenceTransformer('all-mpnet-base-v2')
X_train = model_.encode(texts_train, convert_to_tensor=False)
X_test = model_.encode(texts_test, convert_to_tensor=False)

In [35]:
clf = LogisticRegression(max_iter=1000).fit(X_train, targets_train_)

In [37]:
labels_pred = clf.predict(X_test)

In [38]:
preds_back = ord_enc.inverse_transform(labels_pred.reshape(-1, 1)).flatten()
targets_back = ord_enc.inverse_transform(targets_test_.reshape(-1, 1)).flatten()

In [43]:
print(classification_report(targets_back, preds_back))

                                  precision    recall  f1-score   support

Asset and Reputation damage/loss       1.00      0.03      0.06        32
                     Environment       0.64      0.47      0.54       102
            Information Security       0.00      0.00      0.00         6
                Operational loss       0.00      0.00      0.00        13
                  Process Safety       0.68      0.48      0.56       196
                          Safety       0.84      0.96      0.90       972

                        accuracy                           0.81      1321
                       macro avg       0.53      0.32      0.34      1321
                    weighted avg       0.79      0.81      0.79      1321



### logistic regression 2:

In [44]:
X_train = model.model_body.encode(texts_train, convert_to_tensor=False)
X_test = model.model_body.encode(texts_test, convert_to_tensor=False)

In [45]:
clf = LogisticRegression(max_iter=1000).fit(X_train, targets_train_)

In [46]:
labels_pred = clf.predict(X_test)

In [47]:
preds_back = ord_enc.inverse_transform(labels_pred.reshape(-1, 1)).flatten()
targets_back = ord_enc.inverse_transform(targets_test_.reshape(-1, 1)).flatten()

In [48]:
print(classification_report(targets_back, preds_back))

                                  precision    recall  f1-score   support

Asset and Reputation damage/loss       0.11      0.25      0.16        32
                     Environment       0.59      0.57      0.58       102
            Information Security       1.00      0.33      0.50         6
                Operational loss       0.00      0.00      0.00        13
                  Process Safety       0.70      0.59      0.64       196
                          Safety       0.89      0.90      0.90       972

                        accuracy                           0.80      1321
                       macro avg       0.55      0.44      0.46      1321
                    weighted avg       0.81      0.80      0.81      1321



/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### distill-bert:

In [5]:
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

from torch.utils.data import Dataset, DataLoader

In [6]:
class TextDataset(Dataset):
    
    def __init__(self, ids, masks, y):
        self.ids = ids
        self.masks = masks
        self.y = y
        
        
    def __getitem__(self, index:int):
        return self.ids[index], self.masks[index], self.y[index]

    def __len__(self):
        return self.ids.shape[0]
    
    
class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 7)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1.last_hidden_state 
        
        pooler = torch.mean(hidden_state, dim=1)
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [7]:
texts_train, texts_test, targets_train, targets_test = joblib.load('data_.pkl')

In [8]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(targets_train.reshape(-1, 1))
y_train = enc.transform(targets_train.reshape(-1, 1)).toarray()
y_test = enc.transform(targets_test.reshape(-1, 1)).toarray()

In [9]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

tokens = tokenizer.batch_encode_plus(
            list(texts_train),
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_attention_mask=True
            )

train_ids = tokens['input_ids']
train_masks = tokens['attention_mask']

tokens = tokenizer.batch_encode_plus(
            list(texts_test),
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_attention_mask=True
            )

test_ids = tokens['input_ids']
test_masks = tokens['attention_mask']

In [10]:
y_train = torch.tensor(y_train)
train_ids = torch.tensor(train_ids)
train_masks = torch.tensor(train_masks)

y_test = torch.tensor(y_test)
test_ids = torch.tensor(test_ids)
test_masks = torch.tensor(test_masks)

In [11]:
train_dataset = TextDataset(train_ids, train_masks, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [12]:
test_dataset = TextDataset(test_ids, test_masks, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = DistillBERTClass().to(device)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-05)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
num_epochs = 5
model.train()

for iter_ in range(num_epochs):

    losses = []
    correct = 0

    for ids, masks, y in train_loader:
        ids = ids.to(device)
        masks = masks.to(device)
        y = y.to(device)
        
        y_ = model(ids, masks)
        loss = loss_function(y_, y)
        
        preds = torch.argmax(y_, dim=1)
        true = torch.argmax(y, dim=1)
        correct += (preds == true).sum()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        
    print('###############')
    print(np.mean(losses))
    print((correct / train_dataset.__len__()).item())
        


KeyboardInterrupt



In [42]:
model.eval()
pred_list = []
for ids, masks, y in test_loader:
    ids = ids.to(device)
    masks = masks.to(device)
    y = y.to(device)
    with torch.no_grad():
        y_ = model(ids, masks)
        
    preds = torch.argmax(y_, dim=1)
    preds = preds.cpu().numpy()
    pred_list.append(preds)

In [43]:
preds = np.concatenate(pred_list)
preds = np.eye(7)[preds]

In [44]:
y_test_back = enc.inverse_transform(y_test.numpy()).flatten()
y_pred_back = enc.inverse_transform(preds).flatten()

In [45]:
print(classification_report(y_test_back, y_pred_back))

                                  precision    recall  f1-score   support

Asset and Reputation damage/loss       0.00      0.00      0.00        32
                     Environment       0.10      0.12      0.11       102
            Information Security       0.00      0.00      0.00         6
                Operational loss       0.00      0.00      0.00        13
                  Process Safety       0.17      0.16      0.17       196
                          Safety       0.74      0.77      0.76       972

                        accuracy                           0.60      1321
                       macro avg       0.17      0.18      0.17      1321
                    weighted avg       0.58      0.60      0.59      1321



/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### distillbert -- encode every sentence on it's own

In [5]:
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

from torch.utils.data import Dataset, DataLoader

In [6]:
texts_train, texts_test, targets_train, targets_test = joblib.load('data_.pkl')

In [7]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(targets_train.reshape(-1, 1))
y_train = enc.transform(targets_train.reshape(-1, 1)).toarray()
y_test = enc.transform(targets_test.reshape(-1, 1)).toarray()

In [8]:
def seq_to_batch(sequence_data, batch_size, seq_len):
    shp = tuple(sequence_data.shape)
    batch_data = torch.reshape(sequence_data, [shp[0]*shp[1], *shp[2:]])
    return batch_data

def batch_to_seq(batch_data, batch_size, seq_len):
    shp = tuple(batch_data.shape)
    seq_data = torch.reshape(batch_data, [batch_size, seq_len, *shp[1:]])
    return seq_data

class TextDataset(Dataset):
    
    def __init__(self, ids, masks, y):
        self.ids = ids
        self.masks = masks
        self.y = y
        
        
    def __getitem__(self, index:int):
        return self.ids[index], self.masks[index], self.y[index]

    def __len__(self):
        return self.ids.shape[0]

class SequencesBert(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 7)
       

    def forward(self, input_ids, attention_mask):
        seq_len = input_ids.shape[1]
        batch_size = input_ids.shape[0]
        
        input_ids = seq_to_batch(input_ids, batch_size, seq_len)
        attention_mask = seq_to_batch(attention_mask, batch_size, seq_len)
        
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1.last_hidden_state
        hidden_state = batch_to_seq(hidden_state, batch_size, seq_len)
        hidden_state = torch.mean(hidden_state, dim=[1, 2])
        
        
        pooler = self.pre_classifier(hidden_state)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output
        
        


alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"
multiple_dots = r'\.{2,}'

def split_into_sentences(text):
    
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = [s.strip() for s in sentences]
    if sentences and not sentences[-1]: sentences = sentences[:-1]
    return sentences


def create_sequences(tokens, sentences_max, max_len):
    ids = tokens['input_ids']
    masks = tokens['attention_mask']
    number_missing = sentences_max - len(ids) 
    missing_sequences = [[0] * max_len for i in range(number_missing)]
    ids = ids + missing_sequences
    masks = masks + missing_sequences
    
    return ids, masks
        
    
def pad_empty_sentences(sentences, sentences_max):
    number_missing = sentences_max - len(sentences) 
    sentences = sentences + ['[PAD]'] * number_missing
    
    return sentences


In [9]:
texts_train_segmented = [split_into_sentences(x) for x in texts_train]
texts_test_segmented = [split_into_sentences(x) for x in texts_test]

In [10]:
lens_train = [len(x) for x in texts_train_segmented]
lens_test = [len(x) for x in texts_test_segmented]

In [11]:
sentences_max = max(max(lens_train), max(lens_test))
all_sentences = sum(texts_train_segmented, []) + sum(texts_test_segmented, [])
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
all_lens = [len(tokenizer.encode(x)) for x in all_sentences]
max_len = max(all_lens)

In [12]:
texts_train_segmented = [pad_empty_sentences(x, sentences_max) for x in texts_train_segmented]
texts_test_segmented = [pad_empty_sentences(x, sentences_max) for x in texts_test_segmented]

In [13]:
tokens_ = [tokenizer.batch_encode_plus(
            x,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True
            ) for x in texts_train_segmented]

ids_train = np.stack([x['input_ids'] for x in tokens_])
masks_train = np.stack([x['attention_mask'] for x in tokens_])

tokens_ = [tokenizer.batch_encode_plus(
            x,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True
            ) for x in texts_test_segmented]

ids_test = np.stack([x['input_ids'] for x in tokens_])
masks_test = np.stack([x['attention_mask'] for x in tokens_])



In [14]:
ids_train = torch.tensor(ids_train)
masks_train = torch.tensor(masks_train)
y_train = torch.tensor(y_train)

ids_test = torch.tensor(ids_test)
masks_test = torch.tensor(masks_test)
y_test = torch.tensor(y_test)

In [15]:
train_dataset = TextDataset(ids_train, masks_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

test_dataset = TextDataset(ids_test, masks_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True)

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SequencesBert().to(device)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-05)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
num_epochs = 5
model.train()

for iter_ in range(num_epochs):

    losses = []
    correct = 0

    for ids, masks, y in train_loader:
        ids = ids.to(device)
        masks = masks.to(device)
        y = y.to(device)

        y_ = model(ids, masks)
        
        loss = loss_function(y_, y)
        preds = torch.argmax(y_, dim=1)
        true = torch.argmax(y, dim=1)
        correct += (preds == true).sum()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        
    print('###############')
    print(np.mean(losses))

###############
0.9000213970094202
###############
0.8801893828788157
###############
0.8511827472400245
###############
0.8072080363503064
###############
0.7677551972881888


In [19]:
import joblib
model = model.to('cpu')
joblib.dump(model, 'model.pkl')
joblib.dump(model.state_dict(), 'model_state_dict.pkl')

['model_state_dict.pkl']

In [20]:
model = model.to(device)

In [22]:
model.eval()
pred_list = []
for ids, masks, y in test_loader:
    ids = ids.to(device)
    masks = masks.to(device)
    y = y.to(device)
    with torch.no_grad():
        y_ = model(ids, masks)
        
    preds = torch.argmax(y_, dim=1)
    preds = preds.cpu().numpy()
    pred_list.append(preds)

In [23]:
preds = np.concatenate(pred_list)
preds = np.eye(7)[preds]

In [24]:
y_test_back = enc.inverse_transform(y_test.numpy()).flatten()
y_pred_back = enc.inverse_transform(preds).flatten()

In [25]:
print(classification_report(y_test_back, y_pred_back))

                                  precision    recall  f1-score   support

Asset and Reputation damage/loss       0.00      0.00      0.00        32
                     Environment       0.00      0.00      0.00       102
            Information Security       0.00      0.00      0.00         6
                Operational loss       0.00      0.00      0.00        13
                  Process Safety       0.11      0.06      0.07       196
                          Safety       0.74      0.92      0.82       972

                        accuracy                           0.69      1321
                       macro avg       0.14      0.16      0.15      1321
                    weighted avg       0.56      0.69      0.61      1321



/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
df = pd.DataFrame({'text': texts_test, 'label': y_test_back, 'label_pred': y_pred_back})

In [27]:
joblib.dump(df, 'distill-bert-results.pkl')

['distill-bert-results.pkl']

In [29]:
set(df['label_pred'])

{'Process Safety', 'Safety'}